# Quiz Project Workings and Tests

The purpose of this notebook is to allow for some testing of and workings on processes related to the Quiz project.

The assumed directory/file structure for this notebook is as follows:

- QuizWorkings.ipynb  
- assets/
  - data/
    - ClinicalPracticeWorkbook.txt  

## Table Of Contents

* **0.** [Dependancies and Settings](#0-Dependancies-and-Settings)  
* **1.** [Reading in Quiz Data](#1-Reading-in-Quiz-Data)  
    * **1.1.** [Importing and Defining Structure](#1.1-Importing-and-Defining-Structure)  
    * **1.2.** [Selecting Questions / Randomisation](#1.2-Selecting-Questions-/-Randomisation)  

## 0 Dependancies and Settings

Function used to get names of files in a directory:

In [1]:
# using pathlib as it is cross-platform
from pathlib import Path

def get_filenames(path: str):
    '''
    Returns name of file or all files in directory that path is pointing to.
    
    inputs: string of path to the target directory/file
    outputs: list of string(s) of name(s) of file(s) in target directory/file
    '''
    
    # get the file path from string
    path = Path(path)
    
    # convert to an absolute path
    path = path.absolute()
    
    # store all filename(s) used in a list
    file_list = []
    
    # check if we are referencing a file or directory (folder)
    # if directory, pass through each file and add path to list
    if path.is_dir():
        # get path of each file and append
        for file in path.glob("*"):
            file_list.append(file.name)
    # if file, just add that path to the list
    else:
        # filename is the last part of this path
        filename = path.name
        # rename path to not include the filename
        path = list(path.parents)[0]
        # add filename to list
        file_list.append(filename)

    # sort file_list
    file_list = sorted(file_list)
    
    return file_list

A helper function to filter a list of its elements depending on a boolean list, as seen in numpy array comprehension:

In [2]:
def filter_list(old_list, boolean_filter):
    '''
    Returns a new list of only all the elements in the old 
    list that correspond to a True value in the boolean filter list, along with the corresponding indices
    '''
    
    # new list of only the desired elements of the old list
    new_list = []
    
    # find what indices are True in filter
    filter_indices = [i for i in range(len(boolean_filter)) if boolean_filter[i] == True]
    
    # pass through each index and add to new list
    for index in filter_indices:
        new_list.append(old_list[index])
    
    return new_list, filter_indices

## 1 Reading in Quiz Data

In this section we read in and act on the quiz data from a text file. The text file used here will be the `ClinicalPracticeWorkbook.txt` file.

### 1.1 Importing and Defining Structure

First we import each line in the file (`Path` already imported):

In [3]:
# define path to file
path = Path.cwd() / "assets/data/ClinicalPracticeWorkbook.txt"

# read in data
content = path.read_text(encoding="utf-8")
# split by "\n"
content = content.splitlines()
content

['=Chapter 1',
 '-By law P medicines can only be sold:',
 '+When a counter assistant is present',
 '_When a pharmacist is present',
 '+When a dispenser is present',
 '-Which one of the following is a GSL medicine?',
 '+32 paracetamol 500mg tablets',
 '+30 loperamide capsules',
 '_16 aspirin 300mg tablets',
 '-How long is prescription for a Controlled Drug valid for?',
 '+Six months from the date the doctor signed the prescription',
 '_28 days from the date on the prescription',
 '+28 days from the date the patient handed in the prescription to the pharmacy',
 '-Which of the following is an example of an active ingredient?',
 '_Paracetamol',
 '+Effervescent agent',
 '+Bulking agent',
 '-Which one of the following statements regarding standard operating procedures (SOPs) is TRUE?',
 '_Every step in the dispensing process should have a SOP',
 '+SOPs outline the active ingredients in a tablet',
 '+SOPs are confidential and should not be accessed by pharmacy staff',
 '-Which one of the foll

We can define a quiz structure as follows (where `...` refers to the rest of the string after the symbol identifier, in the actual dictionary, the identifier will not be kept). `line_index` refers to what index in the text file this line resides (counting from `0`). This assumes each question has $2$ wrong answers and $1$ correct one (while `i` and `j` in section and question refers to any number):

- section count: sum i
- section i: 
  - name: '=...'
  - line index: line_index
  - question count: sum j
  - question j:
    - name: '-...'
    - line index: line_index
    - wrong 0:
      - name: '+...'
      - line index: line_index
    - wrong 1:
      - name: '+...'
      - line index: line_index
    - correct:
      - name: '_...'
      - line index: line_index

For example:

In [4]:
quiz = {
    'section count': 1,
    'section 0': {
        'name': 'Chapter 1',
        'line index': 0,
        'question count': 1,
        'question 0': {
            'name': 'By law P medicines can only be sold:',
            'line index': 1,
            'wrong 0': {
                'name': 'When a counter assistant is present',
                'line index': 2,
            },
            'wrong 1': {
                'name': 'When a dispenser is present',
                'line index': 4,
            },
            'correct': {
                'name': 'When a pharmacist is present',
                'line index': 3,
            },
        },
    },
}

quiz

{'section count': 1,
 'section 0': {'name': 'Chapter 1',
  'line index': 0,
  'question count': 1,
  'question 0': {'name': 'By law P medicines can only be sold:',
   'line index': 1,
   'wrong 0': {'name': 'When a counter assistant is present', 'line index': 2},
   'wrong 1': {'name': 'When a dispenser is present', 'line index': 4},
   'correct': {'name': 'When a pharmacist is present', 'line index': 3}}}}

Now let's create this structure. We will pass line-by-line, marking in sections, questions, and answers as we go:

In [5]:
# where all data is stored
quiz = {}
# counters used to keep track of indices
section_counter = -1
question_counter = -1
wrong_counter = -1

# pass through every line
for line_index, line in enumerate(content):
    # get the identifier
    identifier = line[0]
    
    # check if this is a section
    if identifier == '=':
        # increase counter
        section_counter += 1
        
        # reset question counter
        question_counter = -1
        
        # update the section count (index so +1)
        quiz['section count'] = section_counter + 1
        
        # add a section
        quiz[f'section {section_counter}'] = {
            'name': line[1:],
            'line index': line_index,
        }
    
    # check if this is a question
    if identifier == '-':
        # increase counter
        question_counter += 1
        
        # reset wrong counter
        wrong_counter = -1
        
        # update this sections question count (index so +1)
        quiz[f'section {section_counter}']['question count'] = question_counter + 1
        
        # add a question to this section
        quiz[f'section {section_counter}'][f'question {question_counter}'] = {
            'name': line[1:],
            'line index': line_index,
        }
    
    # check if this is a wrong answer
    if identifier == '+':
        # increase counter
        wrong_counter += 1
        
        # add a wrong answer to this question
        quiz[f'section {section_counter}'][f'question {question_counter}'][f'wrong {wrong_counter}'] = {
            'name': line[1:],
            'line index': line_index,
        }
        
    # check if this is a correct answer
    if identifier == '_':
        # add a correct answer to this question
        quiz[f'section {section_counter}'][f'question {question_counter}'][f'correct'] = {
            'name': line[1:],
            'line index': line_index,
        }

quiz

{'section count': 2,
 'section 0': {'name': 'Chapter 1',
  'line index': 0,
  'question count': 10,
  'question 0': {'name': 'By law P medicines can only be sold:',
   'line index': 1,
   'wrong 0': {'name': 'When a counter assistant is present', 'line index': 2},
   'correct': {'name': 'When a pharmacist is present', 'line index': 3},
   'wrong 1': {'name': 'When a dispenser is present', 'line index': 4}},
  'question 1': {'name': 'Which one of the following is a GSL medicine?',
   'line index': 5,
   'wrong 0': {'name': '32 paracetamol 500mg tablets', 'line index': 6},
   'wrong 1': {'name': '30 loperamide capsules', 'line index': 7},
   'correct': {'name': '16 aspirin 300mg tablets', 'line index': 8}},
  'question 2': {'name': 'How long is prescription for a Controlled Drug valid for?',
   'line index': 9,
   'wrong 0': {'name': 'Six months from the date the doctor signed the prescription',
    'line index': 10},
   'correct': {'name': '28 days from the date on the prescription',
  

### 1.2 Selecting Questions / Randomisation

In this section, we will show how to obtain a list of all questions, while randomising the order of them and answers.

First, let's obtain all questions and answers in one list:

In [6]:
# store all question dictionaries in a list
questions = []
# use section count to pass through each section
for section_index in range(quiz['section count']):
    # get each question by using section count to get index
    for question_index in range(quiz[f'section {section_index}']['question count']):
        # append to list
        questions.append(quiz[f'section {section_index}'][f'question {question_index}'])

questions

[{'name': 'By law P medicines can only be sold:',
  'line index': 1,
  'wrong 0': {'name': 'When a counter assistant is present', 'line index': 2},
  'correct': {'name': 'When a pharmacist is present', 'line index': 3},
  'wrong 1': {'name': 'When a dispenser is present', 'line index': 4}},
 {'name': 'Which one of the following is a GSL medicine?',
  'line index': 5,
  'wrong 0': {'name': '32 paracetamol 500mg tablets', 'line index': 6},
  'wrong 1': {'name': '30 loperamide capsules', 'line index': 7},
  'correct': {'name': '16 aspirin 300mg tablets', 'line index': 8}},
 {'name': 'How long is prescription for a Controlled Drug valid for?',
  'line index': 9,
  'wrong 0': {'name': 'Six months from the date the doctor signed the prescription',
   'line index': 10},
  'correct': {'name': '28 days from the date on the prescription',
   'line index': 11},
  'wrong 1': {'name': '28 days from the date the patient handed in the prescription to the pharmacy',
   'line index': 12}},
 {'name': 'W

We can obtain a random question by using:

In [7]:
import random
 
# prints a random value from the list
print(random.choice(questions))

{'name': 'How long is prescription for a Controlled Drug valid for?', 'line index': 9, 'wrong 0': {'name': 'Six months from the date the doctor signed the prescription', 'line index': 10}, 'correct': {'name': '28 days from the date on the prescription', 'line index': 11}, 'wrong 1': {'name': '28 days from the date the patient handed in the prescription to the pharmacy', 'line index': 12}}


And obtaining a random order for answers:

In [8]:
# list of answers
answers = ['wrong 0', 'wrong 1', 'correct']
# randomly shuffle the list
random.shuffle(answers)
answers

['wrong 1', 'correct', 'wrong 0']

Obtaining all questions for one section:

In [9]:
# section to look at
section = 'section 0'

# store all question dictionaries in a list
questions = []
# get each question by using section count to get index
for question_index in range(quiz[section]['question count']):
    # append to list
    questions.append(quiz[section][f'question {question_index}'])

questions

[{'name': 'By law P medicines can only be sold:',
  'line index': 1,
  'wrong 0': {'name': 'When a counter assistant is present', 'line index': 2},
  'correct': {'name': 'When a pharmacist is present', 'line index': 3},
  'wrong 1': {'name': 'When a dispenser is present', 'line index': 4}},
 {'name': 'Which one of the following is a GSL medicine?',
  'line index': 5,
  'wrong 0': {'name': '32 paracetamol 500mg tablets', 'line index': 6},
  'wrong 1': {'name': '30 loperamide capsules', 'line index': 7},
  'correct': {'name': '16 aspirin 300mg tablets', 'line index': 8}},
 {'name': 'How long is prescription for a Controlled Drug valid for?',
  'line index': 9,
  'wrong 0': {'name': 'Six months from the date the doctor signed the prescription',
   'line index': 10},
  'correct': {'name': '28 days from the date on the prescription',
   'line index': 11},
  'wrong 1': {'name': '28 days from the date the patient handed in the prescription to the pharmacy',
   'line index': 12}},
 {'name': 'W